In [4]:
import numpy as np
import pickle  # 用 pickle 替代 cPickle
import gym
np.bool8 = np.bool_

# 超参数
H = 200  # 隐藏层神经元数量
batch_size = 10  # 每多少集进行一次参数更新
learning_rate = 1e-3
gamma = 0.99  # 折扣因子
decay_rate = 0.99  # RMSProp 衰减因子
resume = False  # 是否从以前的检查点恢复
render = False

# 模型初始化
D = 80 * 80  # 输入维度：80x80 网格


model = {}
model['W1'] = np.random.randn(H, D) / np.sqrt(D)  # "Xavier" 初始化
model['W2'] = np.random.randn(H) / np.sqrt(H)

# 更新缓存，用于 RMSProp 平滑梯度
grad_buffer = {k: np.zeros_like(v) for k, v in model.items()}
rmsprop_cache = {k: np.zeros_like(v) for k, v in model.items()}

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))  # sigmoid 函数

def prepro(I):
    """将 210x160x3 uint8 帧预处理为 6400 (80x80) 1D float 向量"""
    I = I[35:195]  # 裁剪
    I = I[::2, ::2, 0]  # 下采样因子为 2
    I[I == 144] = 0  # 删除背景类型 1
    I[I == 109] = 0  # 删除背景类型 2
    I[I != 0] = 1  # 其他设置为 1
    return I.astype(np.float32).ravel()

def discount_rewards(r):
    """对奖励进行折扣"""
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(r.size)):
        if r[t] != 0: running_add = 0  # 如果是游戏边界（仅对 Pong 特有）
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h < 0] = 0  # ReLU
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h  # 返回采取动作 2 的概率和隐藏状态

def policy_backward(eph, epdlogp):
    """反向传播（eph 为隐藏状态数组）"""
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0  # Backprop ReLU
    dW1 = np.dot(dh.T, epx)
    return {'W1': dW1, 'W2': dW2}

# 训练
env = gym.make("Pong-v4")
observation, _ = env.reset()
prev_x = None  # 用于计算差分帧
xs, hs, dlogps, drs = [], [], [], []
running_reward = None
reward_sum = 0
episode_number = 0

while True:
    if render: env.render()

    # 预处理观测，设定网络输入为差分图像
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # 前向传播并采样动作
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3  # 投掷骰子！

    # 记录中间变量（后续用于反向传播）
    xs.append(x)  # 观测值
    hs.append(h)  # 隐藏状态
    y = 1 if action == 2 else 0  # “伪标签”
    dlogps.append(y - aprob)  # 梯度鼓励采取的动作

    # 环境步进，获取新状态
    observation, reward, terminated, truncated, info = env.step(action)  # 更新为新的返回值
    reward_sum += reward

    drs.append(reward)  # 记录奖励

    if terminated or truncated:  # 一个回合结束
        episode_number += 1

        # 堆叠输入、隐藏状态、动作梯度和奖励
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        epdlogp = np.vstack(dlogps)
        epr = np.vstack(drs)
        xs, hs, dlogps, drs = [], [], [], []  # 重置数组

        # 计算折扣奖励
        discounted_epr = discount_rewards(epr)
        # 标准化奖励
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        epdlogp *= discounted_epr  # 使用优势调制梯度
        grad = policy_backward(eph, epdlogp)
        for k in model: grad_buffer[k] += grad[k]  # 在 batch 中累积梯度

        # 每 batch_size 集执行 RMSProp 参数更新
        if episode_number % batch_size == 0:
            for k, v in model.items():
                g = grad_buffer[k]
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g ** 2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v)  # 重置 batch 梯度缓存

        # 记录
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print(f'resetting env. episode reward total was {reward_sum}. running mean: {running_reward}')
        if episode_number % 100 == 0:
            with open('save.p', 'wb') as f:
                pickle.dump(model, f)
        reward_sum = 0
        observation, _ = env.reset()
        prev_x = None

resetting env. episode reward total was -21.0. running mean: -21.0
resetting env. episode reward total was -21.0. running mean: -21.0
resetting env. episode reward total was -21.0. running mean: -21.0
resetting env. episode reward total was -20.0. running mean: -20.99
resetting env. episode reward total was -20.0. running mean: -20.980099999999997
resetting env. episode reward total was -20.0. running mean: -20.970298999999997
resetting env. episode reward total was -19.0. running mean: -20.950596009999998
resetting env. episode reward total was -20.0. running mean: -20.941090049899998
resetting env. episode reward total was -21.0. running mean: -20.941679149400997
resetting env. episode reward total was -21.0. running mean: -20.942262357906987
resetting env. episode reward total was -21.0. running mean: -20.94283973432792
resetting env. episode reward total was -20.0. running mean: -20.933411336984637
resetting env. episode reward total was -20.0. running mean: -20.92407722361479
rese

resetting env. episode reward total was -21.0. running mean: -20.752621948825123
resetting env. episode reward total was -19.0. running mean: -20.735095729336873
resetting env. episode reward total was -21.0. running mean: -20.737744772043506
resetting env. episode reward total was -21.0. running mean: -20.74036732432307
resetting env. episode reward total was -21.0. running mean: -20.74296365107984
resetting env. episode reward total was -21.0. running mean: -20.745534014569042
resetting env. episode reward total was -21.0. running mean: -20.748078674423354
resetting env. episode reward total was -19.0. running mean: -20.73059788767912
resetting env. episode reward total was -21.0. running mean: -20.73329190880233
resetting env. episode reward total was -20.0. running mean: -20.725958989714304
resetting env. episode reward total was -21.0. running mean: -20.72869939981716
resetting env. episode reward total was -21.0. running mean: -20.73141240581899
resetting env. episode reward tota

resetting env. episode reward total was -20.0. running mean: -20.71709348850191
resetting env. episode reward total was -21.0. running mean: -20.71992255361689
resetting env. episode reward total was -21.0. running mean: -20.722723328080722
resetting env. episode reward total was -21.0. running mean: -20.725496094799915
resetting env. episode reward total was -21.0. running mean: -20.728241133851917
resetting env. episode reward total was -21.0. running mean: -20.7309587225134
resetting env. episode reward total was -21.0. running mean: -20.733649135288267
resetting env. episode reward total was -20.0. running mean: -20.726312643935383
resetting env. episode reward total was -21.0. running mean: -20.72904951749603
resetting env. episode reward total was -20.0. running mean: -20.72175902232107
resetting env. episode reward total was -20.0. running mean: -20.714541432097857
resetting env. episode reward total was -20.0. running mean: -20.707396017776876
resetting env. episode reward tota

resetting env. episode reward total was -19.0. running mean: -20.164668211346605
resetting env. episode reward total was -21.0. running mean: -20.17302152923314
resetting env. episode reward total was -18.0. running mean: -20.151291313940806
resetting env. episode reward total was -19.0. running mean: -20.1397784008014
resetting env. episode reward total was -19.0. running mean: -20.128380616793386
resetting env. episode reward total was -18.0. running mean: -20.107096810625453
resetting env. episode reward total was -21.0. running mean: -20.1160258425192
resetting env. episode reward total was -20.0. running mean: -20.114865584094005
resetting env. episode reward total was -20.0. running mean: -20.113716928253066
resetting env. episode reward total was -21.0. running mean: -20.122579758970534
resetting env. episode reward total was -20.0. running mean: -20.121353961380827
resetting env. episode reward total was -21.0. running mean: -20.13014042176702
resetting env. episode reward tota

resetting env. episode reward total was -19.0. running mean: -19.74490804301481
resetting env. episode reward total was -21.0. running mean: -19.757458962584664
resetting env. episode reward total was -19.0. running mean: -19.749884372958817
resetting env. episode reward total was -19.0. running mean: -19.74238552922923
resetting env. episode reward total was -19.0. running mean: -19.73496167393694
resetting env. episode reward total was -19.0. running mean: -19.72761205719757
resetting env. episode reward total was -20.0. running mean: -19.730335936625597
resetting env. episode reward total was -19.0. running mean: -19.723032577259342
resetting env. episode reward total was -17.0. running mean: -19.69580225148675
resetting env. episode reward total was -19.0. running mean: -19.688844228971885
resetting env. episode reward total was -19.0. running mean: -19.681955786682167
resetting env. episode reward total was -16.0. running mean: -19.645136228815346
resetting env. episode reward tot

...

resetting env. episode reward total was 14.0. running mean: 13.323427384024246
resetting env. episode reward total was 12.0. running mean: 13.310193110184002
resetting env. episode reward total was 16.0. running mean: 13.337091179082162
resetting env. episode reward total was 7.0. running mean: 13.27372026729134
resetting env. episode reward total was 12.0. running mean: 13.260983064618426
resetting env. episode reward total was 13.0. running mean: 13.258373233972241
resetting env. episode reward total was 16.0. running mean: 13.28578950163252
resetting env. episode reward total was 13.0. running mean: 13.282931606616195
resetting env. episode reward total was 17.0. running mean: 13.320102290550032
resetting env. episode reward total was 9.0. running mean: 13.276901267644531
resetting env. episode reward total was 13.0. running mean: 13.274132254968086
resetting env. episode reward total was 10.0. running mean: 13.241390932418405
resetting env. episode reward total was 10.0. running me

resetting env. episode reward total was 3.0. running mean: 12.809914561750082
resetting env. episode reward total was 16.0. running mean: 12.841815416132581
resetting env. episode reward total was 17.0. running mean: 12.883397261971256
resetting env. episode reward total was 16.0. running mean: 12.914563289351543
resetting env. episode reward total was 15.0. running mean: 12.935417656458029
resetting env. episode reward total was 13.0. running mean: 12.936063479893448
resetting env. episode reward total was 12.0. running mean: 12.926702845094512
resetting env. episode reward total was 13.0. running mean: 12.927435816643568
resetting env. episode reward total was 17.0. running mean: 12.968161458477132
resetting env. episode reward total was 15.0. running mean: 12.98847984389236
resetting env. episode reward total was 13.0. running mean: 12.988595045453437
resetting env. episode reward total was 16.0. running mean: 13.018709094998902
resetting env. episode reward total was 16.0. running 

resetting env. episode reward total was 19.0. running mean: 12.945056089316973
resetting env. episode reward total was 14.0. running mean: 12.955605528423803
resetting env. episode reward total was 18.0. running mean: 13.006049473139564
resetting env. episode reward total was 11.0. running mean: 12.985988978408168
resetting env. episode reward total was 7.0. running mean: 12.926129088624085
resetting env. episode reward total was 17.0. running mean: 12.966867797737844
resetting env. episode reward total was 14.0. running mean: 12.977199119760467
resetting env. episode reward total was 10.0. running mean: 12.947427128562861
resetting env. episode reward total was 15.0. running mean: 12.967952857277233
resetting env. episode reward total was 8.0. running mean: 12.91827332870446
resetting env. episode reward total was 5.0. running mean: 12.839090595417415
resetting env. episode reward total was 10.0. running mean: 12.810699689463242
resetting env. episode reward total was 15.0. running me

resetting env. episode reward total was 17.0. running mean: 12.561997580282643
resetting env. episode reward total was 7.0. running mean: 12.506377604479816
resetting env. episode reward total was 14.0. running mean: 12.521313828435018
resetting env. episode reward total was 11.0. running mean: 12.506100690150667
resetting env. episode reward total was 13.0. running mean: 12.51103968324916
resetting env. episode reward total was 11.0. running mean: 12.495929286416668
resetting env. episode reward total was 15.0. running mean: 12.520969993552502
resetting env. episode reward total was 12.0. running mean: 12.515760293616976
resetting env. episode reward total was 10.0. running mean: 12.490602690680806
resetting env. episode reward total was 10.0. running mean: 12.465696663773997
resetting env. episode reward total was 11.0. running mean: 12.451039697136256
resetting env. episode reward total was 10.0. running mean: 12.426529300164892
resetting env. episode reward total was 17.0. running 

KeyboardInterrupt: 